# Imports

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from VAE1D import *

In [ ]:
from IPython.core.debugger import set_trace

Define train function in notebook due to import namespace bug.

In [ ]:
def train_VAE1D(dl):
    """
    Execute the training loop
    """
    loss_tracker = AvgTracker()
    kl_tracker = AvgTracker()
    logp_tracker = AvgTracker()
    timer = StopWatch()
    freq = min(log_freq, len(dl))
    
    for i, (X, _) in enumerate(tqdm(dl)):
        
        X = X.to(device)
        timer.lap()  # load time
        
        # Generate transient and compute loss
        X_hat, mu, logvar = model(X)
        loss, loss_desc = criterion(X_hat, X, mu, logvar)
        timer.lap()  # gen time
        
        loss_tracker.update(loss.item())
        kl_tracker.update(loss_desc['KL'].item())
        logp_tracker.update(loss_desc['logp'].item())
        
        if model.training:
            # Update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            timer.lap()  # backprop time
        
        if ((i + 1) % freq == 0) and ((epoch + 1) % ep_freq == 0):
            # Print progress
            if not model.training:
                print('VALIDATION')
            print(f'Epoch: {epoch + 1} ({i + 1}/{len(dl)})')
            print(f'\tData load time: {timer.elapsed[0]:.3f} sec')
            print(f'\tGeneration time: {timer.elapsed[1]:.3f} sec')
            if model.training:
                print(f'\tBackprop time: {timer.elapsed[2]:.3f} sec')
            print(f'\tLog probability: {logp_tracker.val:.4f} '
                  f'(avg {logp_tracker.avg:.4f})')
            print(f'\tKL: {kl_tracker.val:.4f} (avg {kl_tracker.avg:.4f})')
            print(f'\tLoss: {loss_tracker.val:.4f} (avg {loss_tracker.avg:.4f})')

    return loss_tracker.avg, kl_tracker.avg, logp_tracker.avg

# Accumulator

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [ ]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [ ]:
model = VAE1D(size, n_channels, n_latent)

In [ ]:
beta = 1  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [ ]:
desc = 'accumulator'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [ ]:
print(len(train_dl), len(val_dl), len(test_dl))

### Prepare for training

In [ ]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

In [ ]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [ ]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [ ]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [ ]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}-{beta}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')

# Cooler

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [ ]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [ ]:
model = VAE1D(size, n_channels, n_latent)

In [ ]:
beta = 1  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [ ]:
desc = 'cooler'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [ ]:
print(len(train_dl), len(val_dl), len(test_dl))

### Prepare for training

In [ ]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

In [ ]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [ ]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [ ]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [ ]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}-{beta}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')

# Pump

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [ ]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [ ]:
model = VAE1D(size, n_channels, n_latent)

In [ ]:
beta = 1  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [ ]:
desc = 'pump'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [ ]:
print(len(train_dl), len(val_dl), len(test_dl))

### Prepare for training

In [ ]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

In [ ]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [ ]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [ ]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [ ]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}-{beta}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')

# Valve

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [ ]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [ ]:
model = VAE1D(size, n_channels, n_latent)

In [ ]:
beta = 1  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [ ]:
desc = 'valve'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [ ]:
print(len(train_dl), len(val_dl), len(test_dl))

### Prepare for training

In [ ]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

In [ ]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [ ]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [ ]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [ ]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}-{beta}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')